# Auto-Encoding Dictionary Definitions into Consistent Word Embeddings

### Article: [bosc2018auto]
### Repository: [github]


[bosc2018auto]: https://research.fb.com/wp-content/uploads/2018/10/Auto-Encoding-Dictionary-Definitions-into-Consistent-Word-Embeddings.pdf
[github]: https://github.com/tombosc/cpae

### 1. Introduction
1. Word embeddings

1. Similarity vs relatedness

    * Possible to optimize one vs the other, see : [artetxe2018uncovering] (not cited)

1. Post-processing ==> improve existing WE 
    * [faruqui2014retrofitting]

    * [wieting2016charagram] ???
        * Character-based       
            * Trained on paraphrase pairs from the Paraphrase Database (PPDB; Ganitkevitch et al., 2013)
            * looks more like Fasttext ...
            * does not seem to address 'improvement' of WE like [faruqui2014retrofitting]
        * pas rapp, p-e pas la bonne référence 

    * [mrkvsic2017semantic]: post-processing
        * à creuser éventuellement

1. (Re)Generate definition from WE ?
    * [noraset2017definition] (code n/a)
        * RNN
        * WE == fusion of lexical entailment, similarity, relatedness

1. Their model:
    * Definition autoencoder
        * LSTM to encode
        * Decoder  
            * ==> reconstruct BOW representation of definition
        * Consistency penalty / recursivity
            * ==> Proximity of ??!!
                * WE produced by LSTM
                * WE used by LSTM
    * Can also improve existing WE for similarity
                    

[faruqui2014retrofitting]: https://arxiv.org/pdf/1411.4166.pdf
[artetxe2018uncovering]: https://arxiv.org/pdf/1809.02094.pdf
[wieting2016charagram]: https://arxiv.org/pdf/1607.02789.pdf
[mrkvsic2017semantic]: https://arxiv.org/pdf/1706.00374.pdf
[noraset2017definition]: https://arxiv.org/pdf/1612.00394.pdf

### 2. Model
1. Motivation:
    1. objective:
        * recover definitions from WE
    * captures more
        * semantic semilarity (coffee and tea)
        * instead of:
            * relatedness (coffee and cup)
    * single WE per word (no disambiguation in defs)
        * single WE can capture polysemy ???
            * [yaghoobzadeh2016intrinsic]

2. Fonction de coût:
     1. $J(\theta', \theta) = \alpha J_r(\theta', \theta) + \lambda J_p(\theta)$

     2. Consistency penalty :
        1. partie $J_p$ de la fonction de coût, paramétrisé par $\lambda$    

2. Variables:
    1. $\mathcal{V}^{\mathcal{D}} \equiv$ {words used in definitions}
    * $\mathcal{V}^{\mathcal{K}} \equiv$ {words defined}
    * $\lambda \equiv$  c['proximity\_coef'] dans s2sconfigs.py
        1. ex: = 64


[yaghoobzadeh2016intrinsic]: https://arxiv.org/pdf/1606.07902.pdf

* "the decoder can be seen as a __conditional language model__ trained by maximum likelihood to regenerate definition $D_w$ given definition embedding $h = f_θ(D_w)$".

*  To summarize, the consistency penalty has several motivations.
    1. Firstly, it deals with the fact that the recursive process of building representation of words out of definitions does not terminate __????__
    2. Secondly, it is a way to enrich the vocabulary with new words dynamically __????__
    1. Finally, it is a way to integrate prior knowledge in the form of pretrained embeddings __Ok__


* seq2seq:
    * Architecture type des NMT (neural machine translator)
    * Implantation:
        * basé sur les Framework:
            1. 'theano'
            1. 'blocks': concept de 'bricks'
                * une couche au-dessus de Theano, un peu comme Keras
                * permet d'implanter des NN à un niveau plus 'conceptuel', cf: MLP, LSTM, ...
            * 'fuel': gestion des données
        * seq2seq: composé à partir de bricks :
            1. portion encoder:
                1. \_main_lookup ==> blocks.bricks.lookup.LookupTable: [LookupTable], @Pas clair ....
                    * self._main_lookup = LookupTable(self.\_num_input_words, emb_dim, name='main_lookup')
                1. \_encoder_fork ==> blocks.bricks.simple.Linear: [Linear], @transformation simple, i.e.: f(x)=Wx+b

                1. \_encoder_rnn ==> blocks.bricks.recurrent.architectures.LSTM : [LSTM], @@Creuser
            * portion Decoder:
                1. \_translate_layer ==> blocks.bricks.sequences.MLP [MLP]

                1. \_pre_softmax ==> blocks.bricks.simple.Linear [Linear]

                1. \_softmax ==> blocks.bricks.simple.Softmax [Softmax]




[MLP]: https://blocks.readthedocs.io/en/latest/api/bricks.html#blocks.bricks.MLP
[Linear]: https://blocks.readthedocs.io/en/latest/api/bricks.html#blocks.bricks.Linear
[Softmax]: https://blocks.readthedocs.io/en/latest/api/bricks.html#blocks.bricks.Softmax
[LookupTable]: https://blocks.readthedocs.io/en/latest/api/bricks.html#blocks.bricks.lookup.LookupTable
[Linear]: https://blocks.readthedocs.io/en/latest/api/bricks.html#blocks.bricks.Linear
[LSTM]: https://blocks.readthedocs.io/en/latest/api/bricks.html#blocks.bricks.recurrent.architectures.LSTM


    2018-11-19 15:21:43,755: root: INFO: Cost parameters
    ['/seq2seq/encoder_fork.W (300, 1200) trained',
     '/seq2seq/encoder_fork.b (1200,) trained',
     '/seq2seq/encoder_rnn.W_cell_to_forget (300,) trained',
     '/seq2seq/encoder_rnn.W_cell_to_in (300,) trained',
     '/seq2seq/encoder_rnn.W_cell_to_out (300,) trained',
     '/seq2seq/encoder_rnn.W_state (300, 1200) trained',
     '/seq2seq/encoder_rnn.initial_cells (300,) trained',
     '/seq2seq/encoder_rnn.initial_state (300,) trained',
     '/seq2seq/linear.W (300, 1000) trained',
     '/seq2seq/linear.b (1000,) trained',
     '/seq2seq/main_lookup.W (1000, 300) trained',
     '/seq2seq/translate_layer/linear_0.W (300, 300) trained',
     '/seq2seq/translate_layer/linear_0.b (300,) trained']

### 3. Related work
* lexical knowledge
    * intensional definitions
    * genus vs differentia ==> hypernymy
* ....

### 4. Experiments
* Wordnet
2. Similarity/relatedness benchmarks
    * pair of words ranked by human annotators
    * cosine similarity
    * Spearman's correlation ($\rho$) between benchmarks and model results
1. Modèles évalués(Baselines ?):
    * Hill's (see: [hill2015learning])
    * AE (autoencoder)
    * CPAE (... with consistency penalty)


[hill2015learning]: https://arxiv.org/pdf/1504.00548.pdf

### 5. Dictionary only
* Baselines
    * GloVe ...
    * word2vec [mikolov2013distributed]
    * trained on : defined words + definitions
        * (Note: C'est ce que j'avais commencé à regarder pour Enrich/WSA )
* Hill's $\equiv$ n/a
* AE
* CPAE ($\lambda = 8$)
1. CPAE-p ($\lambda = 64$)
    1. 'word2vec embeddings pre-trained on dictionary data'

[mikolov2013distributed]: https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf

#### 5. (suite) Résultats:
* Dans presque tous les cas, CPAE-p ($\lambda = 64$) est significativement mieux que word2vec de base



### 6. Optimisation de Pre-trained embeddings
* Baseline
    * word2vec
        * trained on : Wikipedia dump
* retrofitting [faruqui2014retrofitting]
* dict2vec [tissier2017dict2vec]
* Hill's 
* AE
* CPAE ($\lambda = 4$)


[tissier2017dict2vec]: https://hal-ujm.archives-ouvertes.fr/ujm-01613953/file/emnlp2017.pdf
[faruqui2014retrofitting]: https://arxiv.org/pdf/1411.4166.pdf

# Takeaways

### Observations sur le code
    
* blocks et fuel
    decorators .... 
* frameworks sont complexes
    * Surtout theano
* Theano == symbolic programming
    * "not your grand father's python ..."

### Questions en suspens
* Récurrence des définitions
    * Ça se passe dans les tréfonds de fuel.stream ~~des 'op' de theano~~ 

### Mesure d'évaluation

* Spearman's correlation : [Scipy]
    
    
[Scipy]: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html



### Lien avec la recherche actuelle sur les "language models"


#### Universal Language Models (Pre-calculated models)
1. BERT [devlin2018bert]
1. ELMo [peters2018deep]
1. ULMFiT [howard2018universal]


#### Ma compréhension
1. Différence avec les "Word Embeddings" traditionnels:
    * Architecture du NN pour word2vec beaucoup plus simple.
        * cf: skipgram, CBOW
    * Utiliser un WE existant est relativement simple.
        * Il suffit de calculer la distance entre des mots dans l'espace vectoriel
        * Pour les ULMs c'est plus compliqué:
            * cf: "evaluate_embeddings.sh"/"bin/generate_embeddings.py"


2. Seq2seq: Variations sur le thème des "autoencoders" ??

1. Character-based (vs word based pour les WE) ??  
    * [seq2seq_keras]

[devlin2018bert]: https://arxiv.org/pdf/1810.04805
[howard2018universal]: https://arxiv.org/pdf/1801.06146
[peters2018deep]: https://arxiv.org/pdf/1802.05365

[seq2seq_keras]: https://github.com/Lykkos/enrich/blob/master/_notebooks/seq2seq_keras.ipynb

--- 

--- 

--- 

### For local PDFs

To allow links to local pdf files, need to patch the function

    content_security_policy at line 651 in 'site-packages/notebook/base/handlers.py'
##### cf: /Users/jmpoulin/miniconda2/lib/python2.7/site-packages/notebook/base/handlers.py


from:

    @property
    def content_security_policy(self):
        # In case we're serving HTML/SVG, confine any Javascript to a unique
        # origin so it can't interact with the notebook server.
        return super(AuthenticatedFileHandler, self).content_security_policy + \
                "; sandbox allow-scripts"

to

    @property
    def content_security_policy(self):
        # In case we're serving HTML/SVG, confine any Javascript to a unique
        # origin so it can't interact with the notebook server.
        return super(AuthenticatedFileHandler, self).content_security_policy + \
                ""

See: [Content Security Policy], [Issue 3652]

[Issue 3652]: https://github.com/jupyter/notebook/issues/3652
[Content Security Policy]: https://content-security-policy.com